# Inicio de código

In [1]:
import pandas as pd

In [2]:
pd.options.display.float_format = '${:,.2f}'.format

In [3]:
df = pd.read_csv('/Users/danielmartinez/Desktop/EMTECH/Proyecto_final_2/synergy_logistics_database.csv')
df

,register_id,direction,origin,destination,year,date,product,transport_mode,company_name,total_value
0,1,Exports,Japan,China,2015,31/01/15,Cars,Sea,Honda,33000000
1,2,Exports,Japan,China,2015,01/02/15,Cars,Sea,Honda,16000000
2,3,Exports,Japan,China,2015,02/02/15,Cars,Sea,Honda,29000000
3,4,Exports,Japan,China,2015,03/02/15,Cars,Sea,Honda,14000000
4,5,Exports,Japan,China,2015,04/02/15,Cars,Sea,Honda,17000000
...,...,...,...,...,...,...,...,...,...,...
19051,19052,Imports,Japan,Singapore,2020,27/06/20,Gas turbines,Sea,Union Energy Co,1000000
19052,19053,Imports,Malaysia,Singapore,2020,28/06/20,Gas turbines,Sea,Union Energy Co,2000000
19053,19054,Imports,Malaysia,Singapore,2020,29/06/20,Gas turbines,Sea,Union Energy Co,33000000
19054,19055,Imports,Malaysia,Singapore,2020,30/06/20,Gas turbines,Sea,Union Energy Co,13000000


# Cambio de tipos de variable

In [4]:
df.dtypes

register_id        int64
direction         object
origin            object
destination       object
year               int64
date              object
product           object
transport_mode    object
company_name      object
total_value        int64
dtype: object

In [5]:
df['total_value'] = df['total_value'].astype('float')

In [6]:
df[['direction', 'origin', 'destination', 'product', 'transport_mode', 'company_name']] = df[['direction', 'origin', 'destination', 'product', 'transport_mode', 'company_name']].astype('category')

In [7]:
df['year'] = pd.to_datetime(
    df['year'],
    errors='coerce',
    format = '%m/%d/%Y %H:%M:%S %p'
)

In [8]:
df['date'] = pd.to_datetime(
    df['date'],
    errors='coerce',
    format = '%m/%d/%Y %H:%M:%S %p'
)

In [9]:
df.dtypes

register_id                int64
direction               category
origin                  category
destination             category
year              datetime64[ns]
date              datetime64[ns]
product                 category
transport_mode          category
company_name            category
total_value              float64
dtype: object

# Resolución de problemas

## Opción 1) Rutas de importación y exportación. 

* Synergy logistics está considerando la posibilidad de enfocar sus esfuerzos en las 10 rutas más demandadas. Acorde a los flujos de importación y exportación, ¿cuáles son esas 10 rutas? 

* ¿le conviene implementar esa estrategia? ¿porqué?

In [10]:
"""* Agrupamiento varibales 'origin' y 'destination', contando el número de apariciones.
* Ordenamiento descendente de veces contadas. 
"""
df_grouped = df.groupby(['origin', 'destination']).count()
df_grouped_sorted = df_grouped.sort_values('product', ascending=False)


In [11]:
"""Creación de dataframe con top 10 rutas.
Creación de archivo Excel con dataframe df_top_10_routes.
"""
df_top_10_routes = df_grouped_sorted['register_id'].head(10)
df_top_10_routes.to_excel(r'/Users/danielmartinez/Desktop/EMTECH/Proyecto_final_2/excel/Top_10_routes.xlsx')
df_top_10_routes

origin       destination
South Korea  Vietnam        497
Netherlands  Belgium        437
USA          Netherlands    436
Japan        Mexico         385
China        Mexico         351
             Japan          343
Germany      China          328
Japan        Brazil         306
Germany      France         299
South Korea  Japan          294
Name: register_id, dtype: int64

In [12]:
"""* Agrupamiento varibales 'origin' y 'destination', sumando los datos.
* Ordenamiento descendente de ganancia. 
"""
df_grouped = df.groupby(['origin', 'destination']).sum()
df_grouped_sorted = df_grouped.sort_values('total_value', ascending=False)


In [13]:
"""Creación de dataframe con top 10 rutas por ganancia.
Creación de archivo Excel con dataframe df_top_10_routes_earn.
"""
df_top_10_routes_earn = df_grouped_sorted['total_value'].head(10)
df_top_10_routes_earn.to_excel(r'/Users/danielmartinez/Desktop/EMTECH/Proyecto_final_2/excel/Top_10_routes_earn.xlsx')
df_top_10_routes_earn

origin       destination   
China        Mexico           $12,494,000,000.00
Canada       Mexico            $8,450,000,000.00
South Korea  Vietnam           $6,877,007,000.00
China        Japan             $5,891,000,000.00
Japan        Mexico            $5,829,000,000.00
France       Belgium           $5,538,069,000.00
             United Kingdom    $5,427,000,000.00
China        South Korea       $4,790,000,000.00
South Korea  Japan             $4,741,000,000.00
USA          Mexico            $4,710,000,000.00
Name: total_value, dtype: float64

## Opción 2) Medio de transporte utilizado. 
+ ¿Cuáles son los 3 medios de transporte más importantes para Synergy logistics considerando el valor de las importaciones y exportaciones? 
+ ¿Cuál es medio de transporte que podrían reducir?

In [14]:
"""Agrupamiento por variable transport_mode, sumando los valores y ordenándolos de forma descendente segíun variable
total_value
"""
df_most_used_transport = df.groupby(['transport_mode']).sum().sort_values('total_value', ascending=False)
df_most_used_transport

,register_id,total_value
transport_mode,,
Sea,107111433,"$100,530,622,000.00"
Rail,32521472,"$43,628,043,000.00"
Air,18686086,"$38,262,147,000.00"
Road,23256105,"$33,270,486,000.00"


In [15]:
"""Exportar el dataframe df_most_used_transport a archivo Excel"""
df_most_used_transport.to_excel(r'/Users/danielmartinez/Desktop/EMTECH/Proyecto_final_2/excel/Most_used_transport.xlsx')

## Opción 3) Valor total de importaciones y exportaciones. 

Si Synergy Logistics quisiera enfocarse en los países que le generan el 80% del valor de las exportaciones e importaciones 
* ¿en qué grupo de países debería enfocar sus esfuerzos?

In [16]:
"""Agrupamiento por variables origin, destination y direction, sumando 
y ordenando de forma descendente según la variable total_value"""
df_countries = df.groupby(['origin', 'destination', 'direction']).sum()
df_countries_sorted = df_countries.sort_values('total_value', ascending=False)

In [17]:
"""Creación de columan cumsum que genera la suma acumulada de la variable total_value
"""
df_countries_sorted['cumsum'] = df_countries_sorted['total_value'].cumsum()

In [18]:
"""Creación de la variable percentage que genera el porcentaje acumulado de las ganancias por ruta.
"""
df_countries_sorted['percentage'] = df_countries_sorted['cumsum'] / df_countries_sorted['total_value'].sum() * 100

In [19]:
"""Cambio de forma de expresar los decimales."""
pd.options.display.float_format = '{:,.2f}%'.format
df_80 = df_countries_sorted[['percentage', 'total_value']].head(62).reset_index()
df_80

,origin,destination,direction,percentage,total_value
0,China,Mexico,Exports,5.68%,"12,250,000,000.00%"
1,Canada,Mexico,Exports,9.60%,"8,450,000,000.00%"
2,South Korea,Vietnam,Exports,12.79%,"6,877,007,000.00%"
3,France,Belgium,Exports,15.35%,"5,538,069,000.00%"
4,France,United Kingdom,Exports,17.87%,"5,427,000,000.00%"
...,...,...,...,...,...
57,USA,India,Imports,78.29%,"1,133,000,000.00%"
58,Spain,Russia,Exports,78.79%,"1,085,000,000.00%"
59,India,United Arab Emirates,Exports,79.27%,"1,037,000,000.00%"
60,USA,Netherlands,Exports,79.75%,"1,032,187,000.00%"


In [20]:
df_80[['origin','destination']]

,origin,destination
0,China,Mexico
1,Canada,Mexico
2,South Korea,Vietnam
3,France,Belgium
4,France,United Kingdom
...,...,...
57,USA,India
58,Spain,Russia
59,India,United Arab Emirates
60,USA,Netherlands


In [21]:
df_80_destination = df_80.groupby('destination').sum()
df_80_destination = df_80_destination.drop(['percentage'], axis=1).sort_values('total_value', ascending=False).reset_index()
df_80_destination['cumsum'] = df_80_destination['total_value'].cumsum()
df_80_destination['cumul_percentage'] = df_80_destination['cumsum'] / df_80_destination['total_value'].sum() * 100
df_80_destination.rename(columns={'destination':'countries'}, inplace=True)
df_80_destination.head(12).to_excel(r'/Users/danielmartinez/Desktop/EMTECH/Proyecto_final_2/excel/Top_countries_destination.xlsx')
df_80_destination

,countries,total_value,cumsum,cumul_percentage
0,Mexico,"34,334,000,000.00%","34,334,000,000.00%",19.84%
1,Germany,"13,151,123,000.00%","47,485,123,000.00%",27.44%
2,Thailand,"12,898,000,000.00%","60,383,123,000.00%",34.89%
3,USA,"12,757,040,000.00%","73,140,163,000.00%",42.27%
4,Japan,"11,628,000,000.00%","84,768,163,000.00%",48.98%
5,Belgium,"11,147,211,000.00%","95,915,374,000.00%",55.43%
6,China,"10,059,875,000.00%","105,975,249,000.00%",61.24%
7,United Arab Emirates,"8,345,000,000.00%","114,320,249,000.00%",66.06%
8,United Kingdom,"7,059,000,000.00%","121,379,249,000.00%",70.14%
9,Vietnam,"6,877,007,000.00%","128,256,256,000.00%",74.12%


In [22]:
df_80_origin = df_80.groupby('origin').sum()
df_80_origin = df_80_origin.drop(['percentage'], axis=1).sort_values('total_value', ascending=False).reset_index()
df_80_origin['cumsum'] = df_80_origin['total_value'].cumsum()
df_80_origin['cumul_percentage'] = df_80_origin['cumsum'] / df_80_origin['total_value'].sum() * 100
df_80_origin.rename(columns={'origin':'countries'}, inplace=True)
df_80_origin.head(7).to_excel(r'/Users/danielmartinez/Desktop/EMTECH/Proyecto_final_2/excel/Top_countries_origin.xlsx')
df_80_origin



,countries,total_value,cumsum,cumul_percentage
0,China,"42,371,000,000.00%","42,371,000,000.00%",24.48%
1,USA,"21,988,187,000.00%","64,359,187,000.00%",37.19%
2,France,"18,792,162,000.00%","83,151,349,000.00%",48.05%
3,South Korea,"17,407,007,000.00%","100,558,356,000.00%",58.11%
4,Japan,"17,176,195,000.00%","117,734,551,000.00%",68.04%
5,Germany,"12,743,089,000.00%","130,477,640,000.00%",75.40%
6,Russia,"11,286,000,000.00%","141,763,640,000.00%",81.92%
7,Canada,"8,450,000,000.00%","150,213,640,000.00%",86.80%
8,Singapore,"4,017,000,000.00%","154,230,640,000.00%",89.12%
9,Mexico,"3,508,000,000.00%","157,738,640,000.00%",91.15%


In [23]:
try:   
    !jupyter nbconvert --to python REPORTE_02_MARTINEZ_CORNEJO_DANIEL.ipynb
    # Python se convierte a .py, el script se convierte a .html
         # file_name.ipynb es el nombre del archivo del módulo actual
except:
    pass



[NbConvertApp] Converting notebook REPORTE_02_MARTINEZ_CORNEJO_DANIEL.ipynb to python
[NbConvertApp] Writing 5751 bytes to REPORTE_02_MARTINEZ_CORNEJO_DANIEL.py
